# 🧠 Clasificación y Limpieza de Datos sin errores de ejecución

## 📂 Cargar base de datos

In [1]:
import pandas as pd
import os

ruta = './datos/dataset_ejemplo_1300.csv'
if not os.path.exists(ruta):
    raise FileNotFoundError(f'❌ El archivo no se encuentra en: {ruta}')
df_raw = pd.read_csv(ruta)
df_raw.head()

,ID,Edad,Genero,Ingreso,Nivel_Educativo,Horas_Internet,Ciudad,CodigoID
0,1,22.0,Femenino,4842.42,Secundaria,1.6,Cali,1-Cal
1,2,47.0,Masculino,5742.02,Posgrado,5.6,Medellín,2-Med
2,3,38.0,Otro,4335.12,Técnico,2.3,Villavicencio,3-Vil
3,4,17.0,Otro,4515.57,Tecnólogo,3.2,Bogotá,4-Bog
4,5,28.0,Otro,3846.49,Primaria,1.3,Pasto,5-Pas


## 🎯 Crear variables `Alta_conectividad` y `Nivel_conectividad`

In [2]:
df_raw['Alta_conectividad'] = (df_raw['Horas_Internet'] > 3.5).astype(int)

def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw['Nivel_conectividad'] = df_raw['Horas_Internet'].apply(clasificar_conectividad)

## 🧼 Imputación de variables con `SimpleImputer`

In [3]:
from sklearn.impute import SimpleImputer

df_clean = df_raw.copy()

# Imputar Edad con mediana
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])

# Imputar Ingreso con media
imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

## 🔣 Codificación de variables categóricas

In [4]:
df_clean = pd.get_dummies(df_clean, drop_first=True)

## 📏 Escalado seguro de columnas numéricas

In [5]:
from sklearn.preprocessing import MinMaxScaler

cols_numericas = df_clean.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

## ✅ Filtrar solo columnas numéricas antes del modelo

In [6]:
Xc = df_clean.drop(columns=['Alta_conectividad'])
Xc = Xc.select_dtypes(include=['int64', 'float64'])
yc = df_clean['Alta_conectividad']

## 🤖 Entrenamiento del modelo final sin errores

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)
model_after = LogisticRegression(max_iter=200)
model_after.fit(Xc_train, yc_train)
yc_pred = model_after.predict(Xc_test)
acc_after = accuracy_score(yc_test, yc_pred)
print('✅ Exactitud del modelo:', round(acc_after, 4))

✅ Exactitud del modelo: 1.0
